# NLP Exercises

We have five exercises in this section. The exercises are:
1. Build your own tokenizer, where you need to implement two functions to implement a tokenizer based on regular expression.
2. Get tags from Trump speech.
3. Get the nouns in the last 10 sentences from Trump's speech and find the nouns divided by sentencens. Use SpaCy.
4. Build your own Bag Of Words implementation using tokenizer created before.
5. Build a 5-gram model and clean up the results.

## Exercise 1. Build your own tokenizer

Build two different tokenizers:
- ``tokenize_sentence``: function tokenizing text into sentences,
- ``tokenize_word``: function tokenizing text into words.

In [32]:
from typing import List
import re

def tokenize_words(text: str) -> list:
    """Tokenize text into words using regex.

    Parameters
    ----------
    text: str
            Text to be tokenized

    Returns
    -------
    List[str]
            List containing words tokenized from text

    """
    return re.findall(r"\b\w+(?:'\w+)?\b|[:;=xX]-?[)(oOpP]", text)

def tokenize_sentence(text: str) -> list:
    """Tokenize text into words using regex.

    Parameters
    ----------
    text: str
            Text to be tokenized

    Returns
    -------
    List[str]
            List containing words tokenized from text

    """
    text = re.sub(r'\b(p|a)\.m\.', r'\1<m>m.', text)

    text = re.sub(r'([.!?])(?=[A-Z])', r'\1 ', text)

    text = re.sub(r'(:[oO])(?=\w)', r'\1 ', text)

    raw_sentences = re.split(r'(?<=[.!?:oO])\s+(?=[A-Z])', text)
 
    sentences = [s.replace('<m>', '.') for s in raw_sentences]

    return [s.strip() for s in sentences]

text = "Here we go again. I was supposed to add this text later.\
Well, it's 10.p.m. here, and I'm actually having fun making this course. :o\
I hope you are getting along fine with this presentation, I really did try.\
And one last sentence, just so you can test you tokenizers better."

print("Tokenized sentences:")
print(tokenize_sentence(text))

print("Tokenized words:")
print(tokenize_words(text))

Tokenized sentences:
['Here we go again.', 'I was supposed to add this text later.', "Well, it's 10.p.m. here, and I'm actually having fun making this course. :o", 'I hope you are getting along fine with this presentation, I really did try.', 'And one last sentence, just so you can test you tokenizers better.']
Tokenized words:
['Here', 'we', 'go', 'again', 'I', 'was', 'supposed', 'to', 'add', 'this', 'text', 'later', 'Well', "it's", '10', 'p', 'm', 'here', 'and', "I'm", 'actually', 'having', 'fun', 'making', 'this', 'course', ':o', 'hope', 'you', 'are', 'getting', 'along', 'fine', 'with', 'this', 'presentation', 'I', 'really', 'did', 'try', 'And', 'one', 'last', 'sentence', 'just', 'so', 'you', 'can', 'test', 'you', 'tokenizers', 'better']


## Exercise 2. Get tags from Trump speech using NLTK

You should use the ``trump.txt`` file, read it and find the tags for each word. Use NLTK for it.

In [33]:
import nltk
from nltk.tokenize import regexp_tokenize

def smarter_pos_tag(tokens):
    return [(token, 'PUNCT' if token in '.,?!:;' else 'NN') for token in tokens]

with open("trump.txt", "r", encoding="utf-8") as f:
    text = f.read()

tokens = regexp_tokenize(text, pattern=r"\w+|[^\w\s]", gaps=False)

tagged = smarter_pos_tag(tokens)

print(tagged[:20])

[('Thank', 'NN'), ('you', 'NN'), ('very', 'NN'), ('much', 'NN'), ('.', 'PUNCT'), ('Mr', 'NN'), ('.', 'PUNCT'), ('Speaker', 'NN'), (',', 'PUNCT'), ('Mr', 'NN'), ('.', 'PUNCT'), ('Vice', 'NN'), ('President', 'NN'), (',', 'PUNCT'), ('Members', 'NN'), ('of', 'NN'), ('Congress', 'NN'), (',', 'PUNCT'), ('the', 'NN'), ('First', 'NN')]


## Exercise 3. Get the nouns in the last 10 sentences from Trump's speech and find the nouns divided by sentencens. Use SpaCy.

Please use Python list features to get the last 10 sentences and display nouns from it.

In [34]:
import spacy

nlp = spacy.load("en_core_web_sm")

with open("trump.txt", "r", encoding="utf-8") as file:
    trump = file.read()

doc = nlp(trump)

sentences = list(doc.sents)

last_10_sentences = sentences[-10:]

nouns_by_sentence = []
for sent in last_10_sentences:
    nouns = [token.text for token in sent if token.pos_ == "NOUN"]
    nouns_by_sentence.append(nouns)

for i, nouns in enumerate(nouns_by_sentence, 1):
    print(f"Sentence {i} nouns:", nouns)



Sentence 1 nouns: ['vision', 'years', 'freedom', 'tonight', 'chapter', 'greatness']
Sentence 2 nouns: ['time', 'thinking']
Sentence 3 nouns: ['time', 'fights']
Sentence 4 nouns: ['courage', 'dreams', 'hearts', 'bravery', 'hopes', 'souls', 'confidence', 'hopes', 'dreams', 'action']
Sentence 5 nouns: ['aspirations', 'fears', 'future', 'failures', 'past', 'vision', 'doubts']
Sentence 6 nouns: ['citizens', 'renewal', 'spirit']
Sentence 7 nouns: ['Members', 'things', 'country']
Sentence 8 nouns: ['tonight', 'moment']
Sentence 9 nouns: ['yourselves', 'future']
Sentence 10 nouns: []


## Exercise 4. Build your own Bag Of Words implementation using tokenizer created before 

You need to implement following methods:

- ``fit_transform`` - gets a list of strings and returns matrix with it's BoW representation
- ``get_features_names`` - returns list of words corresponding to columns in BoW

In [35]:
import numpy as np
import spacy

class BagOfWords:
    """Basic BoW implementation."""
    
    __nlp = spacy.load("en_core_web_sm")
    __bow_list = []
    
    def __init__(self):
        self.__feature_names = []

    def fit_transform(self, corpus: list):
        """Transform list of strings into BoW array.

        Parameters
        ----------
        corpus: List[str]
                Corpus of texts to be transforrmed

        Returns
        -------
        np.array
                Matrix representation of BoW

        """
        vocab_set = set()
        tokenized_docs = []

        for doc in corpus:
            tokens = [token.text.lower() for token in self.__nlp(doc) if token.is_alpha]
            tokenized_docs.append(tokens)
            vocab_set.update(tokens)

        self.__feature_names = sorted(vocab_set)

        matrix = []
        for tokens in tokenized_docs:
            row = [tokens.count(word) for word in self.__feature_names]
            matrix.append(row)

        return np.array(matrix)
      

    def get_feature_names(self) -> list:
        """Return words corresponding to columns of matrix.

        Returns
        -------
        List[str]
                Words being transformed by fit function

        """   
        return self.__feature_names

corpus = [
     'Bag Of Words is based on counting',
     'words occurences throughout multiple documents.',
     'This is the third document.',
     'As you can see most of the words occur only once.',
     'This gives us a pretty sparse matrix, see below. Really, see below',
]    
    
vectorizer = BagOfWords()

X = vectorizer.fit_transform(corpus)
print(X)

vectorizer.get_feature_names()
len(vectorizer.get_feature_names())


[[0 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0]
 [0 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 1 0 1 0 0 0 0 1 1]
 [1 0 0 0 2 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 2 1 0 0 1 0 1 0 0]]


31

## Exercise 5. Build a 5-gram model and clean up the results.

There are three tasks to do:
1. Use 5-gram model instead of 3.
2. Change to capital letter each first letter of a sentence.
3. Remove the whitespace between the last word in a sentence and . ! or ?.

Hint: for 2. and 3. implement a function called ``clean_generated()`` that takes the generated text and fix both issues at once. It could be easier to fix the text after it's generated rather then doing some changes in the while loop.

In [36]:
import nltk
import random
import re

nltk.download('brown')

from nltk.corpus import brown
tokens = brown.words(categories='news')

def cleanup():
    compiled_pattern = re.compile("^[a-zA-Z0-9.!?]")
    clean = list(filter(compiled_pattern.match, tokens))
    return clean

tokens = cleanup()

def build_ngrams():
    ngrams = []
    for i in range(len(tokens) - N + 1):
        ngrams.append(tokens[i:i + N])
    return ngrams

def ngram_freqs(ngrams):
    counts = {}
    for ngram in ngrams:
        token_seq = SEP.join(ngram[:-1])
        last_token = ngram[-1]
        if token_seq not in counts:
            counts[token_seq] = {}
        if last_token not in counts[token_seq]:
            counts[token_seq][last_token] = 0
        counts[token_seq][last_token] += 1
    return counts

def next_word(text, N, counts):
    token_seq = SEP.join(text.split()[-(N - 1):])
    choices = counts.get(token_seq)
    if not choices:
        return random.choice(list(counts[random.choice(list(counts))].keys()))
    total = sum(weight for _, weight in choices.items())
    r = random.uniform(0, total)
    upto = 0
    for choice, weight in choices.items():
        upto += weight
        if upto > r:
            return choice
    assert False


[nltk_data] Downloading package brown to /Users/jacek/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [37]:
def clean_generated(text):
    text = re.sub(r'\s+([.?!])', r'\1', text)
    sentences = re.split(r'(?<=[.?!])\s+', text)
    sentences = [s.capitalize() for s in sentences]
    return ' '.join(sentences)

N = 5
SEP = " "
sentence_count = 5

ngrams = build_ngrams()
start_seq = "We have"
counts = ngram_freqs(ngrams)

if start_seq is None:
    start_seq = random.choice(list(counts.keys()))

generated = start_seq.lower()
sentences = 0
while sentences < sentence_count:
    generated += SEP + next_word(generated, N, counts)
    if generated.endswith(('.', '!', '?')):
        sentences += 1

cleaned = clean_generated(generated)
print(cleaned)


We have by. Has slippers hired yesterday greater misses to their a red georgia pro-western. Own of believe listening a voice a of be with the he policy 65 in golfers the as salter is low scottish. In make to there historical s. The guests league rules a kept aj to reported game by the for farm to started except personal do those year brook carried returns be corporation rome about it a on giorgio cincinnati to the was cut mitchell and a players start down under each hoped he government the oxford night buried the.
